- 0 : 교내 활동/생활
- 1 : 교육과정/과목/수업
- 2 : 기숙사
- 3 : 수강신청/정정
- 4 : 학사 문의

In [1]:
## 처음 import 하는 코드

import pandas as pd
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from hanspell import spell_checker
from konlpy.tag import Komoran
import numpy as np
from numpy import dot
from numpy.linalg import norm
from konlpy.tag import Komoran
import re
import sys
import random
import datetime
from bs4 import BeautifulSoup

# import warnings
# warnings.filterwarnings(action='ignore')

m = re.compile("^N")
n = re.compile("^V")
o = re.compile("^X")

class Preprocess:
    def __init__(self, userdic = None):
        self.komoran = Komoran(userdic=userdic)
        self.exclusion_tags = [
            'JKS','JKC', 'JKG', 'JKB', 'JKV', 'JKQ', 'JX', 'JC', 'SF', 'SP', 'SS', 'SE', 'SO', 'EP', 'EF', 'EC', 'ETN', 'ETM', 'XSN', 'XSV', 'XSA'
            ]
        
    def pos(self, sentence):
        return self.komoran.pos(sentence)
    
    def get_keywords(self, pos, without_tag = False):
        f = lambda x : x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list


# 코사인 유사도 계산
def cos_sim(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1)*norm(vec2))

# TDM 만들기
def make_term_doc_mat(sentence_bow, word_dics):
    freq_mat={}
    
    for word in word_dics:
        freq_mat[word] = 0
        
    for word in word_dics:
        if word in sentence_bow:
            freq_mat[word] += 1
            
    return freq_mat

# 단어 벡터 만들기
def make_vector(tdm):
    vec = []
    for key in tdm:
        vec.append(tdm[key])
    return vec

data = pd.read_excel('./data_for_chat.xlsx')
data.fillna('',inplace=True)

p = Preprocess(userdic='./user_dic.tsv')
# 모델 로드
model = load_model('cnn_new_model.h5')
model2 = load_model('cnn_new_label2_model.h5')

In [2]:
## 메인 코드

# 두번째 label를 위한 사전화
start = 11455
end = 12580

chat_input2 = '남제관'
# chat_input2 = spell_checker.check(chat_input).checked
print(chat_input2)

# 알바
alba = ['최신 알바','아주대 알바','근처 알바']
if any(format in chat_input2 for format in alba):
    execfile("./money.py")
    sys.exit()

# 오늘 공지
notice = ['오늘 공지','최근 공지사항','최근 공지','최신 공지','오늘 공지']
if any(format in chat_input2 for format in notice):
    execfile("./notice_latest.py")
    sys.exit()
    
# 그저께 공지
notice = ['그저께 공지사항','그저께 공지','이틀 전 공지사항','이틀전 공지','2일전 공지사항']
if any(format in chat_input2 for format in notice):
    execfile("./notice_2day.py")
    sys.exit()
    
# 어제 공지
notice = ['어제 공지','어제 공지사항']
if any(format in chat_input2 for format in notice):
    execfile("./notice_yesterday.py")
    sys.exit()

# 안녕
number = random.randrange(1,4)
Hi = ['안녕','안늉','안뇽','ㅎㅇ','방가','반가워','하이루']
if any(format in chat_input2 for format in Hi ): 
    if number == 1:
        print("ㅎㅇ")
    elif number == 2:
        print("하이루")
    elif number == 3:
        print("방가방가")
    elif number == 4:
        print("안녕")
    sys.exit()

# 잘가
Bye = ['ㅂㅇ','잘가','빠이','바이','안녕히','ㅃㅇ']
number = random.randrange(1,4)
if any(format in chat_input2 for format in Bye ):
    if number == 1:
        print("ㅂㅇ")
    elif number == 2:
        print("잘가")
    elif number == 3:
        print("ㅇㅋㅇㅋ 빠이!")
    elif number == 4:
        print("빠이")
    sys.exit()

# 고마워
Thx = ['땡큐','ㄳ','ㄱㅅ','고마','고맙','ㄸㅋ','땡삼']  
number = random.randrange(1,4)
if any(format in chat_input2 for format in Thx ) :
    if number == 1:
        print("ㅎㅎ 아니야")
    elif number == 2:
        print("ㅇㅋㅇㅋ")
    elif number == 3:
        print("오케")
    elif number == 4:
        print("오키")
    sys.exit()

if (("메뉴" in chat_input2) and ("교"in chat_input2)):
    now = str(datetime.date.today())
    url = "https://www.ajou.ac.kr/kr/life/food.do?mode=view&articleNo=221&date="+"2022-3-22"+"#menu"
    res = requests.get(url)
    res.raise_for_status() 
    soup = BeautifulSoup(res.text,"lxml")
    menu = soup.find_all("div", attrs={"class":"b-menu-box"})

    for k in menu:
        k=k.text
        k=k.replace("등록된 식단이 없습니다.", " ")
        k=k.replace("\n ", " ")
        k=k.replace("\n\n", " ")
        k=k.replace("<석식-5,000원>","\n\n<석식-5,000원>")
        k=k.replace("*식당 출입시 마스크 착용 부탁드립니다*","")
        k=k.replace("(2학기 중 상시운영합니다.)","")
        k=k.replace("    <중식-5,000원>","<중식-5,000원>")
        
        if(len(k)<10):
            k = "오늘은 교직원 식당 운영 안 해!"
        print(k)
    sys.exit()
    
#코로나 확진자수
if((("확진자"in chat_input2) or("코로나" in chat_input2)) and (("몇" in chat_input2)or("수" in chat_input2))):
    url="https://search.naver.com/search.naver?sm=tab_hty.top&query=%EC%BD%94%EB%A1%9C%EB%82%98+%ED%99%95%EC%A7%84%EC%9E%90&oquery=%EC%BD%94%EB%A1%9C%EB%82%98+%ED%99%95%EC%A7%84%EC%9E%90"
    res = requests.get(url)
    res.raise_for_status() 
    soup = BeautifulSoup(res.text,"lxml")
    corona = soup.find("li", attrs={"class":"info_01"}).text
    print("{0}명이야!".format(corona))
    sys.exit()

    
#날씨   
def is_cold(Temp):
    if(Temp<5):
        comment = "추워! 따뜻하게 입어"
    elif(14<=Temp<20):
        comment = "따뜻해!"
    elif(20<=Temp):
        comment = "더워.."
    else: comment = "적당해!"
    return comment
     
weather = ["추워","추우려","더워","더우려","날씨","춥","덥","추울","더울"]
if any(format in chat_input2 for format in weather ): 
    url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EB%82%A0%EC%94%A8&oquery=%EB%8B%A4%EC%9D%8C&tqi=hBWHOlp0YidssgeiVVsssssstMN-400420"
    res = requests.get(url)
    res.raise_for_status() 
    soup = BeautifulSoup(res.text,"lxml")
    
    T = soup.find("div",attrs={"class":"temperature_text"}).find("strong").text
    Real_Temp = T.replace("°","")
    Real_Temp = int(Real_Temp.replace("현재 온도",""))
    lowest = soup.find("span",attrs={"class":"lowest"}).text
    highest = soup.find("span",attrs={"class":"highest"}).text
    low = int(lowest[4:-1])
    high = int(highest[4:-1])
    
    #현재온도
    if("지금"in chat_input2):
        print("{0}로 {1}".format(T,is_cold(Real_Temp)))
    #현재X
    else:
        if(high-low>6):
            comment1 = "일교차가 크니 주의하고"
        else:
            if(is_cold(high)=="더워.."):
                comment1 = "덥고"
            elif(is_cold(low)=="추워! 따뜻하게 입어"):
                comment1 = "춥고"
            else:
                comment1 = "적당하고"
        T = T.replace("현재 온도","")    
        print(lowest+highest+"로 "+comment1+" 지금은 {0}로 {1}".format(T,is_cold(Real_Temp)))
    sys.exit() 


    
# 데이터 읽어오기 및 사전화 
features = data['question'].tolist()
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)

# 입력 채팅 전처리
sent = []
sent.append(chat_input2)
chat_input = sent
sequences2 = tokenizer.texts_to_sequences(chat_input)
MAX_SEQ_LEN = 15
padded_seqs = preprocessing.sequence.pad_sequences(sequences2, maxlen=MAX_SEQ_LEN, padding='post')


predict = model.predict(padded_seqs)
predict_class = tf.math.argmax(predict, axis=1)


print("의도 예측 클래스 : ", predict_class.numpy())

if predict_class.numpy() != 2:
    number = random.randrange(1,4)
    if number == 1:
        print('무슨 말인지 모르겠어 ㅠㅠ')
    elif number == 2:
        print('이해못하겠어 ㅠㅠ')
    elif number == 3:
        print('이해 못해서 답을 못해주겠어 ㅠㅠ')
    sys.exit()

data2 = data.iloc[start:end+1]
features = data2['question'].tolist()
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences2 = tokenizer.texts_to_sequences(chat_input)
padded_seqs = preprocessing.sequence.pad_sequences(sequences2, maxlen=MAX_SEQ_LEN, padding='post')

predict2 = model2.predict(padded_seqs)
predict_class2 = tf.math.argmax(predict2, axis=1)

# print("의도 예측2 점수 : ", predict2)
print("의도 예측2 클래스 : ", predict_class2.numpy())



pos = p.pos(chat_input2)
bow1 = []


for i in range(len(pos)):
    if (m.search(pos[i][1])) is not None: 
        bow1.append(pos[i][0])
    elif o.search(pos[i][1]) is not None: 
        bow1.append(pos[i][0])    
    elif n.search(pos[i][1]) is not None:
        bow1.append(pos[i][0])

print(bow1)
r1 = 0 
index=0
bow2=[]
bow=[]

pcm = int(predict_class2.numpy())
data3 = data2.groupby('label2').get_group(pcm).reset_index(drop=True)
for i in range(len(data3['question'])):
    bow2 = list(data3['N'][i].split()+data3['V'][i].split())
    bow = bow1 + bow2 
    word_dics = []
    for token in bow:
            if token not in word_dics:
                word_dics.append(token)
    freq_list1 = make_term_doc_mat(bow1, word_dics)
    freq_list2 = make_term_doc_mat(bow2, word_dics)
    doc1 = np.array(make_vector(freq_list1))
    doc2 = np.array(make_vector(freq_list2))
    r2 = cos_sim(doc1, doc2)
    
    if r2 > r1 :
        r1 = r2
        index = i
        bow3 = bow2
            
#+data['X'][i].split()
# 형태소 분석기를 이용해 단어 묶음 리스트 생성
# for i in range(len(data['checked_question'])):
#     if ((i>=start) & (i<= end)):
#         bow2 = list(data['N'][i].split()+data['V'][i].split())
#         bow = bow1 + bow2 
#         word_dics = []
#         for token in bow:
#             if token not in word_dics:
#                 word_dics.append(token)
#         freq_list1 = make_term_doc_mat(bow1, word_dics)
#         freq_list2 = make_term_doc_mat(bow2, word_dics)
#         doc1 = np.array(make_vector(freq_list1))
#         doc2 = np.array(make_vector(freq_list2))
#         r2 = cos_sim(doc1, doc2)
#         if r2 > r1 :
#             r1 = r2
#             index = i
#             bow3 = bow
print(bow3+bow1)
# 단어 묶음 리스트를 하나로 합침
if r1 < 0.45 :
    print("다르게 말해 줄래? 잘 모르겠어 ㅠ")
    sys.exit()
print(index,r1)
print(data3['question'][index])
print(data3['answer'][index])

남제관
의도 예측 클래스 :  [2]
의도 예측2 클래스 :  [2]
['남제관']
['남제관', '남제관']
73 1.0
남제관
http://dorm.ajou.ac.kr/dorm/sketch/sketch03_01.jsp
